<a href="https://colab.research.google.com/github/shaina-12/Artificial-Intelligence/blob/main/Computer%20Vision/Codes/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path_face = "/content/drive/My Drive/Colab Notebooks/Faces/Faces/"

In [ ]:
len_face = len(os.listdir(path_face))

In [ ]:
len_face

2562

In [ ]:
import torchvision
from torchvision import datasets, transforms

In [ ]:
path = []
label = []
for labels in os.listdir(path_face):
  ind = labels.find('_')
  label.append(labels[:ind])
  path.append("/content/drive/My Drive/Colab Notebooks/Faces/Faces/"+labels)

In [ ]:
len(set(label))

31

In [ ]:
path= np.array(path)

In [ ]:
data = pd.DataFrame(path, columns = ['image_file'])

In [ ]:
data['labels'] = label

In [ ]:
data.head()

,image_file,labels
0,/content/drive/My Drive/Colab Notebooks/Faces/...,Henry Cavill
1,/content/drive/My Drive/Colab Notebooks/Faces/...,Hrithik Roshan
2,/content/drive/My Drive/Colab Notebooks/Faces/...,Priyanka Chopra
3,/content/drive/My Drive/Colab Notebooks/Faces/...,Natalie Portman
4,/content/drive/My Drive/Colab Notebooks/Faces/...,Priyanka Chopra


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder =LabelEncoder()             
data['labels'] = label_encoder.fit_transform(data['labels'])

In [ ]:
data.head()

,image_file,labels
0,/content/drive/My Drive/Colab Notebooks/Faces/...,15
1,/content/drive/My Drive/Colab Notebooks/Faces/...,16
2,/content/drive/My Drive/Colab Notebooks/Faces/...,24
3,/content/drive/My Drive/Colab Notebooks/Faces/...,23
4,/content/drive/My Drive/Colab Notebooks/Faces/...,24


In [ ]:
import torch

In [ ]:
transform = transforms.Compose([  
      transforms.ToPILImage(),# Here we do not make data augmentations
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])  # We can simply use this parameter
    ])

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as da
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from skimage import io
class ImageDataset(Dataset):
    def __init__(self,d,transform=None):
        self.annotations = d
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    def __getitem__(self,index):
        image = io.imread(os.path.join(self.annotations.iloc[index, 0]))
        label = torch.tensor(int(self.annotations.iloc[index, 1]))
        if self.transform:
            image = self.transform(image)
        image = torch.reshape(image, (3,224,224))
        return (image, label)

In [ ]:
dataset = ImageDataset(d=data, transform=transform)

In [ ]:
train_size = int(0.80 * len(dataset))
test_size = int(len(dataset) - train_size)
train_set, test_set = torch.utils.data.random_split(dataset,[train_size, test_size])

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [ ]:
model

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

In [ ]:
import torch.nn as nn
model.fc = nn.Linear(model.fc.in_features, 31)

In [ ]:
model = model.cuda()

In [ ]:
len(train_set)

2049

In [ ]:
len(test_set)

513

In [ ]:
batch_size = 3
train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [35]:
num_epochs = 300
for epoch in range(num_epochs):
    
    total_batch = len(train_set)//batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Streaming output truncated to the last 5000 lines.
Epoch [14/50], lter [165/683] Loss: 0.0000
Epoch [14/50], lter [170/683] Loss: 0.0000
Epoch [14/50], lter [175/683] Loss: 0.0000
Epoch [14/50], lter [180/683] Loss: 0.0000
Epoch [14/50], lter [185/683] Loss: 0.0000
Epoch [14/50], lter [190/683] Loss: 0.0000
Epoch [14/50], lter [195/683] Loss: 0.0000
Epoch [14/50], lter [200/683] Loss: 0.0000
Epoch [14/50], lter [205/683] Loss: 0.0000
Epoch [14/50], lter [210/683] Loss: 0.0000
Epoch [14/50], lter [215/683] Loss: 0.0000
Epoch [14/50], lter [220/683] Loss: 0.0000
Epoch [14/50], lter [225/683] Loss: 0.0000
Epoch [14/50], lter [230/683] Loss: 0.0000
Epoch [14/50], lter [235/683] Loss: 0.0000
Epoch [14/50], lter [240/683] Loss: 0.0000
Epoch [14/50], lter [245/683] Loss: 0.0000
Epoch [14/50], lter [250/683] Loss: 0.0000
Epoch [14/50], lter [255/683] Loss: 0.0000
Epoch [14/50], lter [260/683] Loss: 0.0000
Epoch [14/50], lter [265/683] Loss: 0.0000
Epoch [14/50], lter [270/683] Loss: 0.0000
Epo

In [34]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Accuracy of test images: 66.081871 %
